In [1]:
import os
os.environ["SSL_CERT_FILE"] = "/mnt/d/Travel Assistant/Musafir/Fortinet_CA_SSL(15).cer"
os.environ["REQUESTS_CA_BUNDLE"] = "/mnt/d/Travel Assistant/Musafir/Fortinet_CA_SSL(15).cer"

In [2]:
from qdrant_client import QdrantClient, models
import requests
from fastembed import TextEmbedding
import json
import random
import pandas as pd
from tqdm.auto import tqdm


/mnt/d/Travel Assistant/Musafir/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# import the data 
with open('../data/processed_data/documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)


In [4]:
documents[11]

{'city': 'Cairo',
 'section': 'Stay safe',
 'subsection': 'Emergency services',
 'text': 'Ambulance , ☏ 123 .',
 'id': 'bff9ff8c'}

In [5]:
# Ground Truth data
df_gt = pd.read_csv('../data/result/groud-truth-retrieval.csv')

In [6]:
df_gt.head()

,id,city,question
0,f7845786,Cairo,What is the name of the oldest known pyramid i...
1,f7845786,Cairo,Which pyramid in Dahshur has an entrance to th...
2,f7845786,Cairo,What is the distinctive feature of the Bent Py...
3,f7845786,Cairo,How many pyramids are mentioned to be in the D...
4,f7845786,Cairo,What is the atmosphere around Dahshur Pyramids...


In [7]:
ground_truth = df_gt.to_dict(orient='records')

In [8]:
ground_truth[55]

{'id': 'bff9ff8c',
 'city': 'Cairo',
 'question': 'What number should I dial for an ambulance in Cairo?'}

In [9]:
#connecting to local Qdrant instance
qdrant_client = QdrantClient(url="http://localhost:6333")

/tmp/ipykernel_3714/2923977033.py:2: UserWarning: Qdrant client version 1.15.1 is incompatible with server version 1.12.4. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  qdrant_client = QdrantClient(url="http://localhost:6333")


In [10]:
qdrant_client

In [11]:
emb_models = TextEmbedding.list_supported_models()
print(f"There are {len(emb_models)} models supported")
emb_models[0:10]

There are 30 models supported


[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [12]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

## Create collection 

In [13]:
# Define the collection name
collection_name = "traveller-rag"
# Delete a connection 
if qdrant_client.collection_exists(collection_name=collection_name):
    qdrant_client.delete_collection(collection_name=collection_name)
    print(f"Delete the currrent collection {collection_name}")


Delete the currrent collection traveller-rag


In [15]:
# Create the collection with specified vector parameters
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size = 512,
        distance=models.Distance.COSINE
    )
)

## Create, Embed & Insert Points into the Collection


In [20]:
points = []
id = 0

for doc in documents:
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=doc['text'], model=model_handle),
        payload={
            "id": doc['id'],
            "text": doc['text'],
            "city": doc['city'],
            "section": doc['section'],
            "subsection": doc['subsection']

        }
    )

    points.append(point)

    id+=1

In [21]:
qdrant_client.upsert(
    collection_name=collection_name, 
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

# Start the evaluation process

In [22]:
#Hit Rate (HR) or Recall at k
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [23]:
# Mean Reciprocal Rank (MRR)
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [37]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        gt_id = q['id']
        results = search_function(q)

        points = results.points if hasattr(results, "points") else results

        relevance = [
            point.payload.get('id') == gt_id
            for point in points
            if hasattr(point, "payload")
        ]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }


# Semantic Search

## Basline

In [52]:
def qdrant_search(query, limit=5):

    results = qdrant_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        limit=limit,
        with_payload=True
    
    )

    return results

In [53]:
basline_qd_search = evaluate(ground_truth, lambda q: qdrant_search(q['question']))
print(basline_qd_search)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2715/2715 [00:28<00:00, 93.93it/s]

{'hit_rate': 0.8596685082872928, 'mrr': 0.7520933087783905}


return more result improve the evaluation matrix 

## Search with filter


In [40]:
qdrant_client.create_payload_index(
    collection_name=collection_name, 
    field_name="city",
    field_schema="keyword"
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [47]:
def qd_search_filter(query, city, limit):
    results=qdrant_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="city",
                    match=models.MatchValue(value=city)
                )
            ]
        ),
        limit = limit,
        with_payload=True
    )

    return results

In [49]:
filter_qd_search = evaluate(ground_truth, lambda q: qd_search_filter(q['question'], q['city'], 5))
print(filter_qd_search)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2715/2715 [00:27<00:00, 98.98it/s]

{'hit_rate': 0.8674033149171271, 'mrr': 0.7655125844076113}


In [73]:
print(json.dumps(qd_search_filter("What museum I should visit?", "Rome", 5), indent=2))


TypeError: Object of type QueryResponse is not JSON serializable

In [80]:
results = qd_search_filter("What museum I should visit?", "Rome", 5)

points = [
    {
        "score": point.score,
        "payload": point.payload
    }
    for point in results.points
]

print(json.dumps(points, indent=2))

[
  {
    "score": 0.84569573,
    "payload": {
      "id": "1eed9c8a",
      "text": "If you have plenty of time there is absolutely no shortage of other museums covering a wide variety of interests. Examples include the Museum of the Walls (see Rome/South ), the Musical Instrument Museum and a museum devoted to the liberation of Rome from German occupation in the Second World War ( Rome/Esquilino-San Giovanni ).",
      "city": "Rome",
      "section": "See",
      "subsection": "Museums"
    }
  },
  {
    "score": 0.8343532,
    "payload": {
      "id": "18f7911a",
      "text": "If you are in Rome for the Arts there are several world-class museums in the city. The natural starting point is a visit to the area of Villa Borghese in Rome/North Center , where there is a cluster of art museums in and around the Borghese Gardens. Galleria Borghese houses a previously private art collection of the Borghese family, Museo Nazionale di Villa Giulia is home of the world's largest Etruscan ar

In [ ]:
from mistralai import Mistral
from mistralai.models import UserMessage
import os
from dotenv import load_dotenv


In [ ]:
# loads variables from .env
load_dotenv()  

In [ ]:
api_key = os.getenv("API_KEY")

In [ ]:
llm_client = Mistral(api_key = api_key)

In [ ]:
def build_prompt(query, search_results):
    context_template = "Q: {question}\n A: {text}"

    context_parts = []
    for point in search_results.points:  # iterate over .points
        payload = point.payload
        context_parts.append(
            context_template.format(
                question=query,
                text=payload.get("text", "")
            )
        )

    context = "\n\n".join(context_parts)
    
    prompt_template = """
You're a travel assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}

    """.strip()
    
    prompt = prompt_template.format(question=query, context=context)
    return prompt


In [ ]:
def llm(prompt):
    response = llm_client.chat.complete(
        model= "ministral-8b-latest",
        messages=[UserMessage(content=prompt)],
    )

    return response.choices[0].message.content

In [ ]:
def rag(query):
    search_results = qdrant_search(query=query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [ ]:
answer = rag(query)
print(answer)

# hybrid_search

## Sparse vector search with BM25


In [ ]:
import uuid

In [ ]:
# Define the collection name
collection_name = "traveller-sparse"

# Create the collection with specified sparse vector parameters
qdrant_client.create_collection(
    collection_name = collection_name,
    sparse_vectors_config={
        "bm25":models.SparseVectorParams(
            modifier=models.Modifier.IDF
        )
    }
)

In [ ]:
# Send the points to the collection
qdrant_client.upsert(
    collection_name = collection_name,
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "bm25": models.Document(
                    text=doc["text"],
                    model="Qdrant/bm25",
                )
            }, 
            payload={
            "id": doc['id'],
            "text": doc['text'],
            "city": doc['city'],
            "section": doc['section'],
            "subsection": doc['subsection']

            }
        )
        for doc in documents
    ]
)

## Running sparse vector search with BM25

In [ ]:
def bm_search(query: str, limit: int =1) -> list[models.ScoredPoint]:
    results = qdrant_client.query_points(
        collection_name = collection_name, 
        query=models.Document(
            text=query, 
            model="Qdrant/bm25",
        ),
        using="bm25",
        limit=limit,
        with_payload=True
    )
    return results.points

In [ ]:
results = bm_search("Qdrant")
results

In [ ]:
results = bm_search("Roma")
print(results[0].payload['text'])

In [ ]:
results[0].score

In [ ]:
random.seed(202506)
city_piece=random.choice(documents)
print(json.dumps(city_piece, indent=2))

In [ ]:
results = bm_search(city_piece["text"])
print(results[0].payload["text"])

##  Qdrant Universal Query API - prefetching

In [ ]:
# Create the collection with both vector types
collection_name_mx="traveller-sparse-and-dense"
qdrant_client.create_collection(
    collection_name=collection_name_mx,
    vectors_config={
        # Named dense vector for jinaai/jina-embeddings-v2-small-en
        'jina-small':models.VectorParams(
            size=512,
            distance=models.Distance.COSINE
        ),
    },
    sparse_vectors_config={
        "bm25":models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    }
)

In [ ]:
qdrant_client.upsert(
    collection_name=collection_name_mx,
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "jina-small": models.Document(
                    text=doc["text"],
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                "bm25": models.Document(
                    text=doc["text"], 
                    model="Qdrant/bm25",
                ),
            },
            payload={
            "id": doc['id'],
            "text": doc['text'],
            "city": doc['city'],
            "section": doc['section'],
            "subsection": doc['subsection']}

        )
        for doc in documents
    ]
)


In [ ]:
def multi_stage_search(query: str, limit: int = 1) -> list[models.ScoredPoint]:
    results = qdrant_client.query_points(
        collection_name="traveller-sparse-and-dense",
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                using="jina-small",
                # Prefetch ten times more results, then
                # expected to return, so we can really rerank
                limit=(10 * limit),
            ),
        ],
        query=models.Document(
            text=query,
            model="Qdrant/bm25", 
        ),
        using="bm25",
        limit=limit,
        with_payload=True,
    )

    return results.points

In [ ]:
print(json.dumps(city_piece, indent=2))

## Building Hybrid Search

In [ ]:
def rrf_search(query: str, limit: int = 1) -> list[models.ScoredPoint]:
    results = qdrant_client.query_points(
        collection_name="traveller-sparse-and-dense",
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                using="jina-small",
                limit=(5 * limit),
            ),
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="Qdrant/bm25",
                ),
                using="bm25",
                limit=(5 * limit),
            ),
        ],
        # Fusion query enables fusion on the prefetched results
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        with_payload=True,
    )

    return results.points

In [ ]:
results = rrf_search(city_piece["text"])
print(json.dumps(city_piece, indent=2))
print(results[0].payload["text"])

# hybrid-search-and-reranking-es

In [ ]:
import json
import pandas as pd
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

In [ ]:
import numpy as np
np.float_ = np.float64

In [ ]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)